# 🧠 Artificial Neural Network (ANN) for Binary Classification using TensorFlow & TensorBoard

This guide provides an end-to-end pipeline to build an ANN model for binary classification using TensorFlow, including preprocessing, model creation, training with callbacks, and visualization using TensorBoard.

---

## 📌 Step-by-Step Workflow with Code

---

## 1️⃣ Import Required Libraries
```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from datetime import datetime
import os


# 2️⃣ Load Dataset
```python
df = pd.read_csv('your_file.csv')   # 🔹 Replace with your dataset path
X = df.drop('target', axis=1)       # 🔹 Replace 'target' with actual target column
y = df['target']

# 3️⃣ Data Preprocessing
```python
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# 4️⃣ Train-Test Split
```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 5️⃣ Transform Data
```python
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 6️⃣ Build ANN Model
```python
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 7️⃣ Configure Callbacks (TensorBoard & EarlyStopping)
```python
log_dir = os.path.join("logs/fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystop_cb = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 8️⃣ Train Model
```python
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[tensorboard_cb, earlystop_cb],
    verbose=1
)

# 9️⃣ Evaluate Model
```python
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {accuracy:.4f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 🔟 TensorBoard Visualization
```python
print("\n🚀 To visualize training metrics, run the following command in terminal:")
print("tensorboard --logdir logs/fit")
print("Then open the provided URL (usually http://localhost:6006).")